### Funktionen und Import 

In [2]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import pingouin as pg
import pandas as pd
import numpy as np
from config_paths import DATASET_PATH
import scipy.stats as st
import xml.etree.ElementTree as ET 

position_mapping = {
    "TW": "Torwart",
    
    "IVR": "Innenverteidiger",
    "IVZ": "Innenverteidiger",
    "IVL": "Innenverteidiger",
    "RV": "Aussenverteidiger",
    "LV": "Aussenverteidiger",
    "DRM": "Aussenverteidiger",
    "DLM": "Aussenverteidiger",
    
    "DMR": "Defensives Mittelfeld",
    "DMZ": "Defensives Mittelfeld",
    "DML": "Defensives Mittelfeld",
    "RM": "Offensive Aussenbahn",
    "LM": "Offensive Aussenbahn",
    "HR": "Offensives Mittelfeld",
    "HL": "Offensives Mittelfeld",
    "ZO": "Offensives Mittelfeld",
    "ORM": "Offensive Aussenbahn",
    "OLM": "Offensive Aussenbahn",
    
    "RA": "Stürmer",
    "LA": "Stürmer",
    "STR": "Stürmer",
    "STZ": "Stürmer",
    "STL": "Stürmer"
}

def icc_1_1(df, target="full_name", metric="DAS_potential_percentage"):
    
    group_means = df.groupby(target)[metric].mean()
    grand_mean = df[metric].mean()
    n_i = df.groupby(target)[metric].count()
    
    ms_between = ((n_i * (group_means - grand_mean)**2).sum()) / (len(n_i) - 1)
    
    ms_within = df.groupby(target)[metric].apply(lambda x: ((x - x.mean()) ** 2).sum()).sum()
    ms_within /= (len(df) / len(n_i))
    
    k_bar = n_i.mean()
    
    icc = (ms_between - ms_within) / (ms_between + (k_bar - 1) * ms_within)
    
    return icc



def compute_icc(df, metric_column="DAS_potential_percentage"):
    icc = pg.intraclass_corr(
        data=df,
        targets="full_name",
        raters="frame",
        ratings=metric_column
    )
    
    return icc
    

In [3]:
# Import von Data
import_data = pd.read_csv("output/df_player_list.csv")
import_data["position_group"] = import_data["position"].map(position_mapping)
data = import_data[["game_id", "player_id", "frame", "DAS_potential", "DAS_potential_percentage", "id", "full_name", "position", "team", "position_group", "DAS"]].copy()
# data

### ICC pro Spiel

In [4]:
# ICC für J03WQQ
data_stpauli = data[data["game_id"] == "J03WQQ"].copy()
data_regensburg = data[data["game_id"] == "J03WOH"].copy()
data_rostock = data[data["game_id"] == "J03WOY"].copy()
data_nurnberg = data[data["game_id"] == "J03WPY"].copy()
data_kaiserslautern = data[data["game_id"] == "J03WR9"].copy()


# icc_stpauli = icc_1_1(data_stpauli)
# display(f"ICC St Pauli Spiel: {icc_stpauli}") 

# icc_regensburg = icc_1_1(data_regensburg)
# display(f"ICC Regensburg Spiel: {icc_regensburg}") 

# icc_rostock = icc_1_1(data_rostock)
# display(f"ICC Rostock Spiel: {icc_rostock}") 

# icc_nurnberg = icc_1_1(data_nurnberg)
# display(f"ICC Nurnberg Spiel: {icc_nurnberg}") 

# icc_kaiserslautern = icc_1_1(data_kaiserslautern)
# display(f"ICC Kaiserslautern Spiel: {icc_kaiserslautern}") 


# for pos in ["Innenverteidiger", "Aussenverteidiger", "Defensives Mittelfeld"]:
#     df_pos = data_stpauli[data_stpauli["position_group"] == pos]
#     print(pos, icc_1_1(df_pos))
    
min_das = data_stpauli["DAS"].min()
max_das = data_stpauli["DAS"].max()
bases = [min_das, 0.5, 1, 5, 10]
ceils = [0.5, 1, 5, 10, max_das+1]
for base, ceil in zip(bases, ceils):
    df_ceil = data_stpauli[
        (data_stpauli["DAS"] >= base) & (data_stpauli["DAS"] < ceil)
    ]
    # display(icc_1_1(df_ceil))



In [6]:
data_dusseldorf = data_stpauli[data_stpauli["team"] == "Fortuna Düsseldorf"].copy()
data_dusseldorf = data_dusseldorf.groupby("id").filter(lambda x: len(x) > 80)

frames_per_player = data_dusseldorf.groupby("id")["frame"].apply(set)
common_frames = set.intersection(*frames_per_player)
data_dusseldorf = data_dusseldorf[data_dusseldorf["frame"].isin(common_frames)]

icc_result = compute_icc(data_dusseldorf)
display(icc_result)

# icc = icc_1_1(data_dusseldorf)
# display(icc)

# min_das = data_dusseldorf["DAS"].min()
# max_das = data_dusseldorf["DAS"].max()
# bases = [min_das, 0.5, 1, 5, 10]
# ceils = [0.5, 1, 5, 10, max_das+1]

# for base, ceil in zip(bases, ceils):
#     df_ceil = data_dusseldorf[
#         (data_dusseldorf["DAS"] >= base) & (data_dusseldorf["DAS"] < ceil)
#     ]
#     display(ceil, icc_1_1(df_ceil))
#     display(compute_icc(df_ceil))


,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.105421,11.134602,10,935,4.018543e-18,"[0.05, 0.28]"
1,ICC2,Single random raters,0.106444,12.511832,10,850,1.813018e-20,"[0.05, 0.28]"
2,ICC3,Single fixed raters,0.118056,12.511832,10,850,1.813018e-20,"[0.06, 0.3]"
3,ICC1k,Average raters absolute,0.910190,11.134602,10,935,4.018543e-18,"[0.81, 0.97]"
4,ICC2k,Average random raters,0.911069,12.511832,10,850,1.813018e-20,"[0.82, 0.97]"
5,ICC3k,Average fixed raters,0.920076,12.511832,10,850,1.813018e-20,"[0.84, 0.97]"


### Between-Player Seperation (Effect Size)

In [147]:
effect_size = data.groupby("id")["DAS_potential_percentage"].mean().std() / data["DAS_potential_percentage"].std()
S = data.groupby("id")["DAS_potential_percentage"].mean().std()
s = data.groupby("id")["DAS_potential_percentage"].std().mean()
r = 1 - (s / S)**2
# effect_size_qq = data_stpauli.groupby("id")["DAS_potential_percentage"].mean().std() / data_stpauli["DAS_potential_percentage"].std()
display(f"Effect Size über alle hinweg: {effect_size}")
display(f"R über alle hinweg: {r}")
# effect_size_qq

# effect_size_posgroup = data.groupby("position_group")["DAS_potential_percentage"].mean().std() / data["DAS_potential_percentage"].std()
# display(f"Effect Size zwischen Positionsgruppen: {effect_size_posgroup}")

# for pos in data["position_group"].unique():
#     data_view = data[data["position_group"] == pos].copy()
#     value = data_view.groupby("id")["DAS_potential_percentage"].mean().std() / data_view["DAS_potential_percentage"].std()
#     display(f"Effect Size für {pos}: {value}")
min_das = data["DAS"].min()
max_das = data["DAS"].max()
bases = [min_das, 0.5, 1, 5, 10]
ceils = [0.5, 1, 5, 10, max_das+1]

for base, ceil in zip(bases, ceils):
    df_ceil = data[
        (data["DAS"] >= base) & (data["DAS"] < ceil)
    ]
    display(f"Ceil: {ceil}")
    effect_size = df_ceil.groupby("id")["DAS_potential_percentage"].mean().std() / df_ceil["DAS_potential_percentage"].std()
    display(f"Effect Size über alle hinweg: {effect_size}")
    S = df_ceil.groupby("id")["DAS_potential_percentage"].mean().std()
    s = df_ceil.groupby("id")["DAS_potential_percentage"].std().mean()
    r = 1 - (s / S)**2
    display(f"R über alle hinweg: {r}")
    


'Effect Size über alle hinweg: 0.34360616497822144'

'R über alle hinweg: -5.4820952403920025'

'Ceil: 0.5'

'Effect Size über alle hinweg: 0.44864762544824494'

'R über alle hinweg: -2.5968507056045627'

...

'Effect Size über alle hinweg: 0.5444169619674903'

'R über alle hinweg: -1.288913017369675'

'Ceil: 45.30970751482079'

'Effect Size über alle hinweg: 0.5255815249262028'

'R über alle hinweg: -1.0360714461774347'

### Consistency Across Games / Halves

In [279]:
# Korelation zwischen allen Halbzeiten eines Spielers messen

# Frame-Cut für Halbzeit finden
game_frames = data[["game_id", "frame"]].drop_duplicates().sort_values(by=["game_id", "frame"]).reset_index(drop=True)
game_frames["diff"] = game_frames.groupby("game_id")["frame"].diff()
max_diff_per_game = game_frames.groupby("game_id")["diff"].max().reset_index()
half_frames = game_frames[
    (game_frames["game_id"].isin(max_diff_per_game["game_id"])) & (game_frames["diff"]).isin(max_diff_per_game["diff"])
][["game_id", "frame"]]
data_half = data.copy()
data_half = data_half.merge(half_frames, on="game_id", how="left", suffixes=(None, "_half"))

# Halbzeit für Spieler mit mehr als 20 Frames bilden
data_half["period"] = np.where(
    data_half["frame"] < data_half["frame_half"],
    1,
    2
)
data_half = data_half.groupby(["game_id", "id", "period"]).filter(lambda x: len(x) > 20)
data_half = data_half.groupby(["game_id", "period", "player_id", "id", "full_name", "position_group", "team"])["DAS_potential_percentage"].mean().reset_index()
data_half["unit"] = data_half["game_id"] + "_" + data_half["period"].astype(str)

# Standardabweichung über alle Spiele
player_sd = data_half.groupby("id")["DAS_potential_percentage"].std().dropna()  
gesamt_sd = data_half["DAS_potential_percentage"].std()
std_stability = 1 / (1 + player_sd.mean())
std = player_sd.mean()
std_list = 1 / (1 + player_sd)
std_norm = std / gesamt_sd
display(f"Intermatch Stability nach Standardabweichung: {std_stability}")
display(f"Intermatch Stability nach Standardabweichung: {std}")
display(f"Intermatch Stability nach Standardabweichung: {std_norm}")
S = data_half.groupby("id")["DAS_potential_percentage"].mean().std()
s = data_half.groupby("id")["DAS_potential_percentage"].std().mean()
r = 1 - (s / S)**2
display(f"Intermatch Stability nach R: {r}")

# Coefficient of Variation
player_cv = data_half.groupby("id")["DAS_potential_percentage"].std() / data_half.groupby("id")["DAS_potential_percentage"].mean()
inter_match_stability_cv = 1 / (1 + player_cv.mean())
display(f"Intermatch Stability nach CV: {inter_match_stability_cv}")

# Median Absolute Variation
def mad(x):
    return (abs(x - x.median())).median()

player_mad = data_half.groupby("id")["DAS_potential_percentage"].apply(mad)
inter_match_stability_mad = 1 / (1 + player_mad.mean())
display(f"Intermatch Stability nach MAD: {inter_match_stability_mad}")


# Konsistenz zwischen Halbzeiten eines Spiels
df_hz = data_half.pivot(index=["id","game_id"], columns="period", values="DAS_potential_percentage").dropna()
df_hz["difference"] = (df_hz[1] - df_hz[2]).abs()
intra_match_stability = 1 / (1 + df_hz["difference"].mean())
df_hz_var = data_half.groupby(["id", "game_id"])["DAS_potential_percentage"].std()
intra_match_stability_var = 1 / (1 + df_hz_var.mean())


display(f"Intramatch-Stability nach absolut {intra_match_stability} und nach Standardabweichung {intra_match_stability_var}")


'Intermatch Stability nach Standardabweichung: 0.3902804901464385'

'Intermatch Stability nach Standardabweichung: 1.5622597727721068'

'Intermatch Stability nach Standardabweichung: 0.4558458245197084'

'Intermatch Stability nach R: 0.7498544427811821'

'Intermatch Stability nach CV: 0.8215148576787172'

'Intermatch Stability nach MAD: 0.5623308260717502'

'Intramatch-Stability nach absolut 0.3300627421851101 und nach Standardabweichung 0.4106382032052253'

In [273]:
# ICC für Halbzeiten / Spiele

df_half_icc = data_half[data_half["team"] == "Fortuna Düsseldorf"].copy()
df_half_icc["rater"] = df_half_icc["unit"]
df_half_icc["target"] = df_half_icc["id"]
df_half_icc = df_half_icc.rename(columns={"DAS_potential_percentage":"value"})

# Mindestens 10 Halbzeiten => 3 Spieler
units_per_player = df_half_icc.groupby("id")["unit"].apply(set).reset_index()
units_per_player["count"] = units_per_player["unit"].apply(lambda x: len(x))
units_per_player = units_per_player[units_per_player["count"] >= 10]
units = units_per_player["unit"]
common_units = set.intersection(*units)
df_half_icc = df_half_icc[df_half_icc["unit"].isin(common_units)]
df_half_icc = df_half_icc[df_half_icc["id"].isin(units_per_player["id"])]

icc = pg.intraclass_corr(
    data=df_half_icc,
    targets="target",
    raters="rater",
    ratings="value",
    nan_policy="omit"
)
icc_1_1 = icc[icc["Type"] == "ICC1"].iloc[0]["ICC"]
icc_1_1

0.7915457610175858

In [272]:
# ICC für Halbzeiten / Spiele

df_half_icc = data_half[data_half["team"] == "Fortuna Düsseldorf"].copy()
df_half_icc["rater"] = df_half_icc["unit"]
df_half_icc["target"] = df_half_icc["id"]
df_half_icc = df_half_icc.rename(columns={"DAS_potential_percentage":"value"})

# Mindestens 8 Halbzeiten => 8 Spieler (4 Halbzeiten)
units_per_player = df_half_icc.groupby("id")["unit"].apply(set).reset_index()
units_per_player["count"] = units_per_player["unit"].apply(lambda x: len(x))
units_per_player = units_per_player[units_per_player["count"] >= 8]
units = units_per_player["unit"]
common_units = set.intersection(*units)

df_half_icc = df_half_icc[df_half_icc["unit"].isin(common_units)]
df_half_icc = df_half_icc[df_half_icc["id"].isin(units_per_player["id"])]

icc = pg.intraclass_corr(
    data=df_half_icc,
    targets="target",
    raters="rater",
    ratings="value",
    nan_policy="omit"
)
icc_1_1 = icc[icc["Type"] == "ICC1"].iloc[0]["ICC"]
icc_1_1

0.8094783486543558

### Convergent Validity

In [78]:
# Tracking Daten extrahieren
def extract_tracking_events_from_xml(xml_path, game_id):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    rows = []
    
    def get_attr(elem, name, default=None):
        return elem.attrib[name] if name in elem.attrib else default
    
    for event in root.iter("Event"):
        event_time = get_attr(event, "EventTime")
        x = float(get_attr(event, "X-Position", "0"))
        y = float(get_attr(event, "Y-Position", "0"))
        event_id = get_attr(event, "EventId")
        
        
        for elem in event.iter():
            tag = elem.tag
            if tag in ["BallClaiming"]:
                player_id = get_attr(elem, "Player")
                type = get_attr(elem, "Type")
                rows.append([game_id, player_id, "BallClaiming", type])
                
            if tag in ["BlockedShot"]:
                player_id = get_attr(elem, "Player")
                type = get_attr(elem, "GoalPrevented")
                rows.append([game_id, player_id, "BlockedShot", type])
            
            if tag == "TacklingGame":
                winner = get_attr(elem, "Winner")
                winner_role = get_attr(elem, "WinnerRole")
                possession_change = get_attr(elem, "PossessionChange") == "true"
                type = get_attr(elem, "Type")
                
                if possession_change and winner_role == "withoutBallControl":
                    rows.append([game_id, winner,  "WonTackle", type])
                    
            if tag == "OtherBallAction":
                action = get_attr(elem, "DefensiveClearance") == "true"
                player = get_attr(elem, "Player")
                if action:
                    rows.append([game_id, player, "clearance"])
    df = pd.DataFrame(rows)      
    return df


In [ ]:
# Vergleich Korrelation mit Event-Metriken
game_ids = data.copy()["game_id"].unique()
dataframes = []

for game_id in game_ids:
    df = extract_tracking_events_from_xml(DATASET_PATH / f"{game_id}/event_data.xml", game_id)
    dataframes.append(df)

event_data = pd.concat(dataframes)
event_data = event_data.rename(columns={0: "game_id", 1: "player_id", 2: "action", 3: "type"})

event_counts = event_data.groupby(["game_id","player_id"])["action"].value_counts().unstack(fill_value=0).reset_index()
defense_cols = [col for col in event_counts.columns if col not in ["game_id", "player_id"]]
event_counts["def_event_score"] = event_counts[defense_cols].sum(axis=1)

das_stats = (
    data.groupby(["game_id", "id", "position_group"])["DAS_potential_percentage"]
        .mean()
        .reset_index()
        .rename(columns={"id": "player_id"})
)
merged = das_stats.merge(event_counts, on=["game_id", "player_id"], how="left")
merged = merged.fillna(0)
rank_directions = {
    "DAS_potential_percentage": True,
    "BallClaiming": False,
    "BlockedShot": False,
    "WonTackle": False,
    "clearance": False,
    "def_event_score": False,
}
for col, asc in rank_directions.items():
    merged[col + "_rank"] = merged[col].rank(ascending=asc)

# display(merged[["def_event_score", "DAS_potential_percentage", "def_event_score_rank", "DAS_potential_percentage_rank"]])
# merged["DAS_score_norm"] =  -merged["DAS_potential_percentage"]

corr_raw = merged[["BallClaiming",  "BlockedShot", "WonTackle", "clearance", "def_event_score", "DAS_potential_percentage"]].corr(method="spearman", numeric_only=True)["DAS_potential_percentage"]
display("Spearman Korrelation (Werte):")
display(corr_raw)

corr_ranked = merged[["BallClaiming_rank",  "BlockedShot_rank", "WonTackle_rank", "clearance_rank", "def_event_score_rank", "DAS_potential_percentage_rank"]].corr(method="spearman", numeric_only=True)["DAS_potential_percentage_rank"]
display("\nSpearman Korrelation (Rankings):")
display(corr_ranked)
# corr_ranked = merged.rank().corr(method="spearman", numeric_only=True)["DAS_score_norm"]
# display("\nSpearman Korrelation (Rankings):")
# display(corr_ranked)




,def_event_score,DAS_potential_percentage,def_event_score_rank,DAS_potential_percentage_rank
0,10.0,6.810055,13.5,95.0
1,1.0,1.902828,152.0,8.0
2,9.0,8.321236,19.0,132.0
3,1.0,13.264924,152.0,205.0
4,0.0,5.037593,192.5,54.0
...,...,...,...,...
212,0.0,7.029652,192.5,105.0
213,6.0,10.524775,45.5,175.0
214,3.0,11.470733,98.0,187.0
215,7.0,8.728392,32.5,143.0


'Spearman Korrelation (Werte):'

BallClaiming                0.176037
BlockedShot                 0.336764
WonTackle                   0.432432
clearance                   0.426517
def_event_score             0.436931
DAS_potential_percentage    1.000000
Name: DAS_potential_percentage, dtype: float64

'\nSpearman Korrelation (Rankings):'

BallClaiming_rank               -0.176037
BlockedShot_rank                -0.336764
WonTackle_rank                  -0.432432
clearance_rank                  -0.426517
def_event_score_rank            -0.436931
DAS_potential_percentage_rank    1.000000
Name: DAS_potential_percentage_rank, dtype: float64

In [146]:
# Vergleich Korrelation mit Spieler Bewertungen
df_ratings = pd.read_csv("../externe_scores.csv", delimiter=";", decimal=",")
positions = das_stats["position_group"].unique()
merge_ratings = das_stats.merge(df_ratings, left_on=["game_id", "player_id"], right_on=["game_id", "id"], how="left")
merge_ratings = merge_ratings[merge_ratings["id"].notna()]
merge_ratings
# display(merge_ratings)

# corr_raw = merge_ratings[["DAS_potential_percentage", "SofaScore Rating", "WhoScored Rating"]].corr(method="spearman")["DAS_potential_percentage"]
# display("Spearman Korrelation (Werte):")
# display(corr_raw)

for pos in positions:
    if pos in ["Offensives Mittelfeld", "Stürmer", "Offensive Aussenbahn", "Torwart"]:
        continue
    df_pos = merge_ratings[merge_ratings["position_group"] == pos]
    corr_raw = df_pos[["DAS_potential_percentage", "SofaScore Rating", "WhoScored Rating"]].corr(method="spearman")["DAS_potential_percentage"]
    display(f"Spearman Korrelation (Werte) für {pos}:")
    display(corr_raw)
    
    corr_rank = df_pos[["DAS_potential_percentage", "SofaScore Rating", "WhoScored Rating"]].rank().corr(method="spearman")["DAS_potential_percentage"]
    display(f"Spearman Korrelation (Rang) für {pos}:")
    display(corr_rank)


# df_pos = merge_ratings[merge_ratings["position_group"] == "Innenverteidiger"]
# das = merge_ratings.loc[:, ["DAS_potential_percentage"]]
# sofa = merge_ratings.loc[:, ["SofaScore Rating"]]
# who = merge_ratings.loc[:, ["WhoScored Rating"]]
# das_sofa = das.corrwith(sofa, method="spearman").mean()
# das_who = das.corrwith(who, method="spearman").mean()
# sofa_who = sofa.corrwith(who, method="spearman").mean()
# display(f"Stability Das und Ratings: {das_sofa} | {das_who} und zwischen ratings: {sofa_who}")



'Spearman Korrelation (Werte) für Aussenverteidiger:'

DAS_potential_percentage    1.000000
SofaScore Rating           -0.032875
WhoScored Rating            0.059565
Name: DAS_potential_percentage, dtype: float64

'Spearman Korrelation (Rang) für Aussenverteidiger:'

DAS_potential_percentage    1.000000
SofaScore Rating           -0.032875
WhoScored Rating            0.059565
Name: DAS_potential_percentage, dtype: float64

'Spearman Korrelation (Werte) für Defensives Mittelfeld:'

...

DAS_potential_percentage    1.000000
SofaScore Rating           -0.259356
WhoScored Rating           -0.114358
Name: DAS_potential_percentage, dtype: float64

'Spearman Korrelation (Werte) für Innenverteidiger:'

DAS_potential_percentage    1.000000
SofaScore Rating           -0.164088
WhoScored Rating           -0.124323
Name: DAS_potential_percentage, dtype: float64

'Spearman Korrelation (Rang) für Innenverteidiger:'

DAS_potential_percentage    1.000000
SofaScore Rating           -0.164088
WhoScored Rating           -0.124323
Name: DAS_potential_percentage, dtype: float64

In [ ]:
# Vergleich Korrelation mit Event-Metriken
game_ids = data.copy()["game_id"].unique()
dataframes = []

for game_id in game_ids:
    df = extract_tracking_events_from_xml(DATASET_PATH / f"{game_id}/event_data.xml", game_id)
    dataframes.append(df)

event_data = pd.concat(dataframes)
event_data = event_data.rename(columns={0: "game_id", 1: "player_id", 2: "action", 3: "type"})

event_counts = event_data.groupby(["game_id","player_id"])["action"].value_counts().unstack(fill_value=0).reset_index()
defense_cols = [col for col in event_counts.columns if col not in ["game_id", "player_id"]]
event_counts["def_event_score"] = event_counts[defense_cols].sum(axis=1)

das_stats = (
    data.groupby(["game_id", "id", "position_group"])["DAS_potential_percentage"]
        .mean()
        .reset_index()
        .rename(columns={"id": "player_id"})
)
merged = das_stats.merge(event_counts, on=["game_id", "player_id"], how="left")
merged = merged.fillna(0)
rank_directions = {
    "DAS_potential_percentage": True,
    "BallClaiming": False,
    "BlockedShot": False,
    "WonTackle": False,
    "clearance": False,
    "def_event_score": False,
}
for col, asc in rank_directions.items():
    merged[col + "_rank"] = merged[col].rank(ascending=asc)

# display(merged[["def_event_score", "DAS_potential_percentage", "def_event_score_rank", "DAS_potential_percentage_rank"]])
# merged["DAS_score_norm"] =  -merged["DAS_potential_percentage"]

corr_raw = merged[["BallClaiming",  "BlockedShot", "WonTackle", "clearance", "def_event_score", "DAS_potential_percentage"]].corr(method="spearman", numeric_only=True)["DAS_potential_percentage"]
display("Spearman Korrelation (Werte):")
display(corr_raw)

corr_ranked = merged[["BallClaiming_rank",  "BlockedShot_rank", "WonTackle_rank", "clearance_rank", "def_event_score_rank", "DAS_potential_percentage_rank"]].corr(method="spearman", numeric_only=True)["DAS_potential_percentage_rank"]
display("\nSpearman Korrelation (Rankings):")
display(corr_ranked)
# corr_ranked = merged.rank().corr(method="spearman", numeric_only=True)["DAS_score_norm"]
# display("\nSpearman Korrelation (Rankings):")
# display(corr_ranked)




,def_event_score,DAS_potential_percentage,def_event_score_rank,DAS_potential_percentage_rank
0,10.0,6.810055,13.5,95.0
1,1.0,1.902828,152.0,8.0
2,9.0,8.321236,19.0,132.0
3,1.0,13.264924,152.0,205.0
4,0.0,5.037593,192.5,54.0
...,...,...,...,...
212,0.0,7.029652,192.5,105.0
213,6.0,10.524775,45.5,175.0
214,3.0,11.470733,98.0,187.0
215,7.0,8.728392,32.5,143.0


'Spearman Korrelation (Werte):'

BallClaiming                0.176037
BlockedShot                 0.336764
WonTackle                   0.432432
clearance                   0.426517
def_event_score             0.436931
DAS_potential_percentage    1.000000
Name: DAS_potential_percentage, dtype: float64

'\nSpearman Korrelation (Rankings):'

BallClaiming_rank               -0.176037
BlockedShot_rank                -0.336764
WonTackle_rank                  -0.432432
clearance_rank                  -0.426517
def_event_score_rank            -0.436931
DAS_potential_percentage_rank    1.000000
Name: DAS_potential_percentage_rank, dtype: float64

### Discriminant Validity

In [7]:
anova = pg.anova(dv="DAS_potential_percentage", between="position_group", data=data)
display(anova)

kruskal = pg.kruskal(data=data, dv="DAS_potential_percentage", between="position_group")
display(kruskal)

,Source,ddof1,ddof2,F,p-unc,np2
0,position_group,6,15051,204.20334,2.683968e-251,0.075277


,Source,ddof1,H,p-unc
Kruskal,position_group,6,1832.747885,0.0


### Situation Validity

In [8]:
### Positionsdaten bereitstellen
game_ids = data["game_id"].unique()
frame_ids = data["frame"].unique()
tracking_files = list(DATASET_PATH.rglob("*tracking_data.parquet"))
dataframes = []

for file in tracking_files:
    df = pd.read_parquet(file)
    parts = file.parts
    df["game_id"] = parts[9]
    df = df[df["frame"].isin(frame_ids)]
    dataframes.append(df)

tracking_wide = pd.concat(dataframes, ignore_index=True)
tracking_long = tracking_wide.melt(
    id_vars=["game_id", "frame", "ball_x", "ball_y"],
    var_name="player_coord",
    value_name="coord"
)
tracking_long[["player_id", "axis"]] = tracking_long["player_coord"].str.extract(r"(home_\d+|away_\d+)_(x|y)")
tracking_long = tracking_long.pivot_table(
    index=["game_id", "frame", "player_id", "ball_x", "ball_y"],
    columns="axis",
    values="coord"
).reset_index()
tracking_long

axis,game_id,frame,player_id,ball_x,ball_y,x,y
0,J03WMX,10051,away_10,2.40,-3.33,-4.58,21.16
1,J03WMX,10051,away_11,2.40,-3.33,-10.77,-31.75
2,J03WMX,10051,away_2,2.40,-3.33,13.88,0.46
3,J03WMX,10051,away_25,2.40,-3.33,-10.77,-13.62
4,J03WMX,10051,away_27,2.40,-3.33,44.16,0.03
...,...,...,...,...,...,...,...
207552,J03WR9,176575,home_33,-53.17,-3.26,-22.21,0.73
207553,J03WR9,176575,home_46,-53.17,-3.26,-21.72,-5.7
207554,J03WR9,176575,home_5,-53.17,-3.26,-24.53,0.99
207555,J03WR9,176575,home_8,-53.17,-3.26,-24.15,5.02


In [9]:
merged = data.merge(tracking_long, on=["game_id", "frame", "player_id"], how="left")
merged["dist_ball"] = np.sqrt(((merged["x"] - merged["ball_x"])**2 + (merged["y"] - merged["ball_y"])**2).astype(float))
merged["near_ball"] = merged["dist_ball"] < 12
merged_filtered = merged[merged["near_ball"]]

# => Was mache ich jetzt damit?
merged["zone"] = pd.cut(merged["x"], bins=[-54,-18,18,54], labels=["defensive","middle","offensive"])
display(pg.anova(dv="DAS_potential_percentage", between="zone", data=merged))
display(merged.groupby("near_ball")["DAS_potential_percentage"].describe())

nb = merged[merged["near_ball"]]["DAS_potential_percentage"]
fb = merged[~merged["near_ball"]]["DAS_potential_percentage"]
test = st.mannwhitneyu(nb, fb, alternative="two-sided")
display(test)


def cliffs_delta(x, y):
    from itertools import product
    greater = sum(a > b for a, b in product(x, y))
    smaller = sum(a < b for a, b in product(x, y))
    return (greater - smaller) / (len(x) * len(y))

delta = cliffs_delta(nb.values, fb.values)
print("Cliff's delta:", delta)



,Source,ddof1,ddof2,F,p-unc,np2
0,zone,2,15055,0.043862,0.957086,0.000006


,count,mean,std,min,25%,50%,75%,max
near_ball,,,,,,,,
False,11675.0,7.097711,9.369715,0.0,0.796436,3.735588,9.878815,89.971065
True,3383.0,10.834916,11.762594,0.0,1.859187,6.959304,15.688823,81.503421


MannwhitneyuResult(statistic=23584084.5, pvalue=1.5814046766023978e-66)

Cliff's delta: 0.19423592328692207


### Archiv

In [ ]:
# Halbzeiten vergleichen:
# df_pivot = data_half.pivot(index=["full_name"], columns=["game_id", "period"], values="DAS_potential_percentage")
# display(df_pivot)
# hz1 = df_pivot.xs(1, level="period", axis=1)
# hz2 = df_pivot.xs(2, level="period", axis=1)
# stability = hz1.corrwith(hz2, method="spearman").mean()
# display("Stability across halves over all games:", stability)

# stability = df_pivot.T.corr(method="spearman").mean()

# display("Stability across over all games:", stability.head())